# Automated testing techniques

## Arrange, Act, Assert

In the absense of a compelling reason to do otherwise (which is rare), each test case should test exactly one thing, clearly express what claim about the code under test it is testing, and test that claim and that claim only. Often it is even feasible to name the test case with a name that will be read by humans as making that claim (e.g., `test_widgets_are_nontoxic`). A test case should:

1. **Arrange.** Zero or more actions that set up for what is being tested. Some or all of these actions may be done in test *fixtures* rather than in the test cases themselves, but they are done in preparation for an individual test case.

2. **Act.** An action that exercises the code under test, whose effects will be checked.

3. **Assert.** An assertion about the effect of the action.

Arrange-Act-Assert is sometimes regarded to be incomplete. Arguably a fourth kind of test logic deserves to be distinguished explicitly:

4. **Cleanup.** Zero or more actions that tear something down that was used for testing, freeing resources or restoring an invariant. Some or all of these actions may be done in test fixtures rather than in the test cases themselves, but they are done to clean up after an individual test case.

These steps should always be clearly identifiable in your test code, and the test code within a test case should not be written out of order, except in the rare situations where that is not feasible.

You should always strongly consider separating all these steps so that each statement does only one of those things. (You may still often need to have multiple statements that do one of them, of course.) For example, if you are going to call a function and assert something about the value it returns, you should strongly consider assigning the result to a variable, then using that variable in an assertion on a subsequent line.

Controversial claim: In some situations, particularly very simple ones with a large amount of repetition, it may be defensible to combine the steps so that, for example, a statement is both acting and asserting (or, more rarely, both arranging and acting).

- If you do this, you should always strongly consider taking the more orthodox approach of strictly separating them, understand why you are deciding not to do that, and be able to defend your decision to a hypothetical interlocutor.

- Some people say this should never be done.

- Do not do it if it makes tests even slightly less clear, harder to read and understand, more complicated, or harder to verify for correctness.

It should be readily apparent to anyone reading a test what logic in that test is arrangement, what logic is acting, and what logic is asserting. It is only defensible to choose to do more than one of these kinds of things in the same statement if you are confident that doing so does not make this distinction any less clear to anyone reading the code.

As an opposite approach, some people write `# Arrange`, `# Act`, and `# Assert` comments identifying separate sections of their tests. This is not wrong. But I suggest against doing it, or at least against do it *habitually*, because that information should always be readily apparent in the *code* of the test. It is possible to write complicated or unclear tests where the distinction between arrangement, acting, and asserting is not clear even if the code is separated out. If you find you're doing that, you should redesign the test, if possible. If you do separate the sections and comment them, you should make sure the distinctions would still be fully clear even without the comments.

**A test case should almost always have exactly one assertion.** In particular, if you have more than one assertion, it is a strong sign you should be writing more than one test. One solution can be to parameterize the test.

Subtests are a form of parameterization and, if separate assertions are done in separate subtests, then the rule to have exactly one assertion is not violated (though you should still make sure you understand why you have decided to use subtests rather than separately written tests or some other form of parameterization).

Sometimes it is not feasible to avoid having multiple assertions, and even not feasible to place them in separate subtests. For example, it may be that two claims must be asserted separately for the test code to clearly express what those claims are, but that those claims are very closely tied, such that it is impossible or very misleading to test the second one unless the first one has been found to hold.  In such a situation, you may need to write a test with multiple assertions. But make sure:

1. *That the first claim really is part of what you are trying to test and make an assertion about.* If instead the second claim is the one the test is really about, and the first claim is just to verify that the preconditons necessary to test the second claim have been established, then you should check for the first claim and cause the test to *error out* (rather than merely failing). The best way to do this may vary by testing framework, but often you can do it by checking for the precondition with an `if` statement and raising an exception directly. This is one of the rare situations where it it can be reasonable to raise `Exception` (rather than a more derived exception type). Make sure your exception message clearly states the reason for the error.

2. *That it would actually be wrong, or at least clearly undesirable, to test the second claim if the first has failed.* If it would always be acceptable to test both, then even if you cannot reasonably write separate tests or parameterize the whole test case, you can likely use subtests (assuming your testing framework supports subtests or you have extended its functionality to do so, such as with a plugin).

## Hiding classes from `unittest`

In [1]:
import unittest

Three ways to keep a `unittest` test runner from picking up test classes:

#### 1. Delete the names (variables) that refers to the base classes, afterwards.

In [2]:
class _BaseClassForSomeTests(unittest.TestCase):
    ...  # Blah blah, shared tests.

class TestWidgets(_BaseClassForSomeTests):
    ...  # Anything else we need for widget tests.

class TestGadgets(_BaseClassForSomeTests):
    ...  # Anything else we need for gadget tests.

del _BaseClassForSomeTests
# TestWidgets and TestGadgets are still derived classes of _BaseClassForSomeTests.

#### 2. Nest the base classes inside another class (created for that purpose).

In [3]:
class _Bases:
    class SharedWidgetGadgetTests(unittest.TestCase):
        ... # Blah blah, shared tests.

class TestWidgets(_Bases.SharedWidgetGadgetTests):
    ...  # Anything else we need for widget tests.

class TestGadgets(_Bases.SharedWidgetGadgetTests):
    ... # Anything else we need for gadget tests.

#### 3. Put the base classes in a separate module the test runner won't find.

We might have file `base_test_classes.py`, so the module name doesn't start with `test` (and to be safe, some test runners, in some configurations, look for `test` at the end, too, so avoid that). In that module:

In [4]:
class BaseClassForSomeTests(unittest.TestCase):
    ...  # Blah blah, shared tests.

Then one or more other actual test modules (`test_whatever.py`) can import that module or import classes from it:

```python
from base_test_classes import BaseClassForSomeTests

class TestWidgets(BaseClassForSomeTests):
    ...  # Anything else we need for widget tests.
    
class TestGadgets(BaseClassForSomeTests):
    ...  # Anything else we need for gadget tests.
```

This approach is the least commonly done, and is mostly only reasonable when two or more separate test modules would benefit from using the base classes, or when it would otherwise be desirable to have the tests in a separate module (even if only one other module is going to use them).